## GOOGLE MAPS DATA SCRAPPING

###  Extracting The Busniss's Name, Adress, Phon Number & Website Of Each Business.

> ### Inputs Cell.

In [1]:
search_subject = "warehouses"
search_location = 'Texas, USA'
no_of_results = 50

> ### Dependecies Cell.

In [2]:
url ="https://www.google.com"
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Firefox
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.events import EventFiringWebDriver, AbstractEventListener
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
Firefox_options = Options()
#Firefox_options.add_argument("--disable-extensions")
Firefox_options.add_argument("--disable-gpu")
# Firefox_options.add_argument("--headless")
Firefox_options.add_argument("--start-maximized")
# Firefox_options.headless = True # also works
import time
from time import sleep
import pandas as pd
import numpy as np
from tqdm import tqdm

> ### Navigating To The Businesses Page . 

In [3]:
start_time = time.time()

# driver = webdriver.Firefox()
driver = webdriver.Firefox(options=Firefox_options)
actions = ActionChains(driver)

driver.get(url)
# set the language to english
set_language = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.LINK_TEXT, 'English')))
set_language.click()


search_bar = driver.find_element(By.CSS_SELECTOR, 'textarea[title="Search"]')
search_bar.click()

search_bar.send_keys(search_subject + ' in ' + search_location)
print(search_subject + ' in ' + search_location)

find = driver.find_element(By.CSS_SELECTOR, 'input[value="Google Search"]')#(By.LINK_TEXT, 'Google Search')
find.click()

businesses_results_E = driver.find_element(By.CSS_SELECTOR, 'div[class="iNTie"]')
businesses_results = businesses_results_E.find_element(By.TAG_NAME, 'g-more-link')
businesses_results.click()

result_list =[]

warehouses in Texas, USA


> ### Scraper Code Cell. 

In [4]:

with tqdm(total=no_of_results,unit ="result", desc ="scraper") as pbar:
    while no_of_results > len(result_list):
        
        driver.switch_to.default_content()
        ActionChains(driver).send_keys(Keys.HOME).perform()# to get back to the top of the page after going to the next page
        sleep(3)
        results_container = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, 'div[class="rlfl__tls rl_tls"]')))  # results container element 
        primary_info = results_container.find_elements(By.CSS_SELECTOR,'div[jscontroller="AtSb"]')# REMOVE THE S IN ELEMENTS AFTER DONE TESTING
        
        sleep(3)
        for P in primary_info: # iterates through the results

            if no_of_results == len(result_list):
                break
            
            business_E_found = "no"
            while business_E_found == "no":     
                try:
                    sleep(.05)
                    business_E_0 = P.find_element(By.CSS_SELECTOR, 'div[class="rllt__details"]')
                    business_E = business_E_0.find_element(By.CSS_SELECTOR, 'span[class="OSrXXb"]')# business name
                    business = business_E.text
                    business_E.click()
                    end_of_the_page = driver.find_element(By.ID, 'bottomads') 
                    business_E_found = "yes"
                except:
                    business_E_found = "no"
            
            info_card_E_0 = driver.find_element(By.CSS_SELECTOR, 'div[jscontroller="Mv8snb"]')
            info_card_E_1= info_card_E_0.find_element(By.CLASS_NAME, "h2yBfgNjGpc__inline-item-view")
            sleep(2.5)
            
            info_card_E= info_card_E_1.find_element(By.CLASS_NAME, 'immersive-container')
            
            
            try:
                location = info_card_E.find_element(By.CLASS_NAME,"LrzXr")     
                address = location.text
                if len(address) < 3:
                    address = location.get_attribute("innerHTML")
            except:
                address= "not available"
                
            try:    
                website = info_card_E.find_element(By.CSS_SELECTOR, 'a[class="dHS6jb"]').get_attribute('href')
            except:
                website = "not available"
                

            try:
                ratting = info_card_E_1.find_element(By.CLASS_NAME,'Aq14fc').text
            except:
                ratting = "not available"
                
            if len(ratting) == 0:
                try:
                    ratting = info_card_E.find_elements(By.CSS_SELECTOR, 'div[jstcache="134"]')    
                    for r in ratting:
                        try:
                            ratting = str(int(r.text))
                            break
                        except:
                            pass
                except:
                    ratting = "not available"    
                
            
                                
            try:
                phone = info_card_E.find_element(By.CSS_SELECTOR, 'a[jscontroller="LWZElb"]').text
            except:
                phone = "not available"
                
            if len(phone)<3 or len(phone) == 0:
                try:
                    phone = info_card_E.find_elements(By.CSS_SELECTOR, 'div[jstcache="134"]')    
                    for p in phone:
                        try:
                            phone = str(int(p.text[-4:-1]))
                            break
                        except:
                            pass
                except:
                    phone = "not available"
                    
                
            result = {"Business Name": business, 
                    "Phone Number" :phone,
                    "Ratting": ratting, 
                    "Address": address, 
                    "Website": website
                    }  
            
            result_list.append(result)
            
            try:
                close_card = info_card_E_1.find_element(By.CSS_SELECTOR ,'div[role="button"]')
                close_card.click()
            except:
                pass
            
            business_E_found = "no"
            
            pbar.update(1)
            
        if no_of_results > len(result_list):
            next_page = driver.find_elements(By.CSS_SELECTOR, 'table[role="presentation"] td')[-1]
            next_page.click()
    
        
main_df = pd.DataFrame(result_list)     
driver.quit()

T = str((time.time() - start_time)/60)
minutes = T[:T.index(".")]
seconds = str(int(T[T.index(".")+1:])*60)[:2]
Time =  minutes+"m"+seconds +"s."

print(f"done!!\n{len(result_list)} results scraped successfully in:",Time)


scraper: 100%|██████████| 50/50 [02:58<00:00,  3.58s/result]


done!!
50 results scraped successfully in: 3m17s.


In [5]:
main_df = pd.DataFrame(result_list)

In [6]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Business Name  50 non-null     object
 1   Phone Number   50 non-null     object
 2   Ratting        50 non-null     object
 3   Address        50 non-null     object
 4   Website        50 non-null     object
dtypes: object(5)
memory usage: 2.1+ KB


In [7]:
main_df.tail()

,Business Name,Phone Number,Ratting,Address,Website
45,Amazon Fullfillment Center DFW7,+1 866-216-1072,3.7,"700 Westport Pkwy, Fort Worth, TX 76177, Unite...",https://www.amazon.jobs/
46,UPS Warehouse 7597,+1 800-877-6947,3.9,"2320 E Bardin Rd, Arlington, TX 76018, United ...",not available
47,Amazon LBB5,+1 806-747-1711,4.3,"1110 E Hunter St, Lubbock, TX 79403, United St...",not available
48,Amazon FTW2,+1 866-216-1072,4.1,"2701 W Bethel Rd, Coppell, TX 75019, United St...",not available
49,DEMATIC CD WAREHOUSE,not available,2.4,"11200 Harmon Rd, Fort Worth, TX 76177, United ...",not available


> ### Saving The Output file as csv.

In [8]:
main_df.to_csv(f"G-maps {search_subject} in {search_location} {len(result_list)} results .csv", index= False)